# Chapter 7: Capstone -- Writing a Complete Song

## Learning Objectives

By the end of this chapter you will be able to:

1. **Combine all concepts** from Chapters 1-6 into a single songwriting pipeline
2. Generate a complete song with **structure, melody, chords, rhythm, and lyric framework**
3. Visualize the entire song as a **multi-dimensional map**
4. Listen to the result as synthesized audio

---

## The Plan

We will build a song step by step, using the tools from every previous chapter:

```
Step 1: Song Structure      (Ch 1)  →  Choose a form, define sections
Step 2: Chord Progression   (Ch 3)  →  Pick chords for each section
Step 3: Melody              (Ch 2)  →  Generate melody over the chords
Step 4: Rhythm              (Ch 4)  →  Create a drum pattern
Step 5: Lyric Framework     (Ch 5)  →  Define rhyme scheme + syllable targets
Step 6: Arrangement         (Ch 6)  →  Layer instruments across sections
Step 7: Full Song Map       (All)   →  Visualize everything together
```

All functions are defined inline so this notebook is completely self-contained.

In [ ]:
# === FOUNDATION: Note and chord utilities (from Chapters 2 & 3) ===

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap
from dataclasses import dataclass, field
from collections import defaultdict
from IPython.display import Audio, display

NOTE_NAMES: list[str] = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

def note_to_midi(name: str, octave: int = 4) -> int:
    return NOTE_NAMES.index(name) + (octave + 1) * 12

def midi_to_freq(midi: int) -> float:
    return 440.0 * 2 ** ((midi - 69) / 12)

def midi_to_name(midi: int) -> str:
    return f"{NOTE_NAMES[midi % 12]}{midi // 12 - 1}"

# Scale and chord definitions
SCALES: dict[str, list[int]] = {
    "Major":            [2, 2, 1, 2, 2, 2, 1],
    "Natural Minor":    [2, 1, 2, 2, 1, 2, 2],
    "Major Pentatonic":  [2, 2, 3, 2, 3],
}

CHORD_TYPES: dict[str, list[int]] = {
    "major": [0, 4, 7], "minor": [0, 3, 7], "diminished": [0, 3, 6],
}

DIATONIC_QUALITIES: list[str] = ["major", "minor", "minor", "major", "major", "minor", "diminished"]
ROMAN_NUMERALS: list[str] = ["I", "ii", "iii", "IV", "V", "vi", "vii°"]

def build_scale(root_midi: int, pattern: list[int], octaves: int = 1) -> list[int]:
    notes = [root_midi]
    for _ in range(octaves):
        for interval in pattern:
            notes.append(notes[-1] + interval)
    return notes

def build_chord(root_midi: int, chord_type: str = "major") -> list[int]:
    return [root_midi + i for i in CHORD_TYPES[chord_type]]

def diatonic_chords(root_name: str, octave: int = 4) -> list[tuple[str, str, list[int]]]:
    root_midi = note_to_midi(root_name, octave)
    scale = [root_midi]
    for interval in SCALES["Major"]:
        scale.append(scale[-1] + interval)
    chords = []
    suffix_map = {"major": "", "minor": "m", "diminished": "dim"}
    for i in range(7):
        root = scale[i]
        quality = DIATONIC_QUALITIES[i]
        notes = build_chord(root, quality)
        cname = f"{NOTE_NAMES[root % 12]}{suffix_map[quality]}"
        chords.append((ROMAN_NUMERALS[i], cname, notes))
    return chords

print("Foundation utilities loaded.")
print(f"Diatonic chords in G major: {', '.join(c[1] for c in diatonic_chords('G'))}")

## Step 1: Song Structure

We will write a song in the standard **verse-chorus** form. Our song:

- Key: **G major**
- Tempo: **110 BPM**
- Form: **Intro - Verse - Chorus - Verse - Chorus - Bridge - Chorus - Outro**

In [ ]:
# === STEP 1: Define the song structure (Chapter 1) ===

@dataclass
class SongSection:
    name: str
    kind: str       # verse, chorus, bridge, intro, outro
    bars: int
    chord_degrees: list[int]   # indices into diatonic chords (repeated to fill bars)
    
KEY = "G"
TEMPO = 110
TIME_SIG = (4, 4)

# Define the song's sections
song_structure: list[SongSection] = [
    SongSection("Intro",     "intro",   4,  [0, 4, 5, 3]),       # I - V - vi - IV
    SongSection("Verse 1",   "verse",   8,  [0, 4, 5, 3] * 2),  # I-V-vi-IV x2
    SongSection("Chorus 1",  "chorus",  8,  [3, 0, 4, 5] * 2),  # IV-I-V-vi x2
    SongSection("Verse 2",   "verse",   8,  [0, 4, 5, 3] * 2),
    SongSection("Chorus 2",  "chorus",  8,  [3, 0, 4, 5] * 2),
    SongSection("Bridge",    "bridge",  8,  [1, 4, 0, 0, 1, 4, 3, 3]),  # ii-V-I-I-ii-V-IV-IV
    SongSection("Chorus 3",  "chorus",  8,  [3, 0, 4, 5] * 2),
    SongSection("Outro",     "outro",   4,  [0, 4, 5, 3]),
]

total_bars = sum(s.bars for s in song_structure)
duration_sec = total_bars * TIME_SIG[0] * 60 / TEMPO

print(f"Song: Key of {KEY} major, {TEMPO} BPM, {TIME_SIG[0]}/{TIME_SIG[1]} time")
print(f"Total: {total_bars} bars ({duration_sec:.0f} seconds / {duration_sec/60:.1f} minutes)")
print()
for s in song_structure:
    bar_vis = '█' * s.bars
    print(f"  {s.name:<12} [{s.kind:<7}] {bar_vis} ({s.bars} bars)")

## Step 2: Chord Progression

In [ ]:
# === STEP 2: Realize chord progressions for each section (Chapter 3) ===

all_diatonic = diatonic_chords(KEY, octave=3)  # octave 3 for bass range

print(f"Diatonic chords in {KEY} major:")
for rn, cname, notes in all_diatonic:
    print(f"  {rn:<6} {cname:<6} {[midi_to_name(n) for n in notes]}")

print("\nChord progressions per section:")
for section in song_structure:
    chords = [all_diatonic[d][1] for d in section.chord_degrees]
    # Show unique pattern (first N chords before repeat)
    pattern_len = len(set(zip(section.chord_degrees, range(len(section.chord_degrees)))))
    display_chords = chords[:min(4, len(chords))]  # show first 4
    repeat = "(x2)" if len(chords) > 4 else ""
    print(f"  {section.name:<12} | {' | '.join(display_chords)} | {repeat}")

## Step 3: Melody Generation

In [ ]:
# === STEP 3: Generate melody for each section (Chapter 2) ===

def generate_section_melody(
    scale_notes: list[int],
    chord_degrees: list[int],
    notes_per_bar: int = 4,
    step_bias: float = 0.7,
    seed: int = 42,
) -> list[int]:
    """Generate a melody that follows the chord progression.
    
    The melody starts on a chord tone for each bar, then
    walks through the scale with occasional leaps.
    """
    rng = np.random.default_rng(seed)
    melody = []
    
    for degree in chord_degrees:
        # Get the chord tones for this bar
        chord_root = all_diatonic[degree][2][0]  # root note of chord
        # Find the closest scale note to the chord root in our range
        closest_idx = min(range(len(scale_notes)),
                         key=lambda i: abs(scale_notes[i] - chord_root))
        
        # Start on a chord tone
        idx = closest_idx
        bar_melody = [scale_notes[idx]]
        
        for _ in range(notes_per_bar - 1):
            if rng.random() < step_bias:
                step = rng.choice([-1, 1])
            else:
                step = rng.choice([-2, 2])
            idx = max(0, min(len(scale_notes) - 1, idx + step))
            bar_melody.append(scale_notes[idx])
        
        melody.extend(bar_melody)
    
    return melody

# Build scale for melody (octaves 4-5 for vocal range)
melody_scale = build_scale(note_to_midi(KEY, 4), SCALES["Major"], octaves=2)

# Generate melodies for each section with different seeds for variety
section_melodies: dict[str, list[int]] = {}
seeds = {"intro": 10, "verse": 42, "chorus": 77, "bridge": 99, "outro": 10}

for section in song_structure:
    seed = seeds[section.kind]
    step_bias = 0.75 if section.kind in ("verse", "intro", "outro") else 0.55  # chorus/bridge = more leaps
    melody = generate_section_melody(
        melody_scale, section.chord_degrees,
        notes_per_bar=4, step_bias=step_bias, seed=seed
    )
    section_melodies[section.name] = melody

# Display first 8 notes of each section's melody
print("Melody preview (first 8 notes):")
for section in song_structure:
    notes = section_melodies[section.name][:8]
    names = [midi_to_name(n) for n in notes]
    print(f"  {section.name:<12} {' '.join(names)}")

In [ ]:
# Plot the melodic contour for verse vs. chorus

fig, axes = plt.subplots(2, 1, figsize=(14, 6), sharex=False)
fig.suptitle("Melodic Contour: Verse vs. Chorus", fontsize=14, fontweight="bold")

for ax, section_name, color in zip(axes, ["Verse 1", "Chorus 1"], ["#AA96DA", "#FCBAD3"]):
    melody = section_melodies[section_name]
    beats = list(range(1, len(melody) + 1))
    ax.plot(beats, melody, "o-", color=color, linewidth=1.5, markersize=4, alpha=0.8)
    ax.fill_between(beats, min(melody) - 1, melody, alpha=0.15, color=color)
    
    # Mark bar lines
    for bar in range(0, len(melody), 4):
        ax.axvline(x=bar + 1, color="gray", linewidth=0.3, alpha=0.5)
    
    ax.set_ylabel("MIDI Pitch")
    ax.set_title(section_name, loc="left", fontsize=11)
    ax.grid(True, alpha=0.2)

axes[-1].set_xlabel("Beat")
plt.tight_layout()
plt.show()

## Step 4: Rhythm Pattern

In [ ]:
# === STEP 4: Drum patterns for each section type (Chapter 4) ===

DrumPattern = dict[str, list[int]]

def make_pattern(kick: str, snare: str, hihat: str) -> DrumPattern:
    def parse(s: str) -> list[int]:
        return [1 if c.lower() == 'x' else 0 for c in s.replace(" ", "")]
    return {"Kick": parse(kick), "Snare": parse(snare), "Hi-hat": parse(hihat)}

# Different patterns for different energy levels
drum_patterns: dict[str, DrumPattern] = {
    "verse": make_pattern(
        kick  = "x...........x...",
        snare = "....x.......x...",
        hihat = "x.x.x.x.x.x.x.x",
    ),
    "chorus": make_pattern(
        kick  = "x...x...x...x...",
        snare = "....x.......x...",
        hihat = "x.xxx.xxx.xxx.xx",
    ),
    "bridge": make_pattern(
        kick  = "x...............x...............",  # sparse, half-time feel
        snare = "........x...............x.......",
        hihat = "x...x...x...x...x...x...x...x..",
    ),
    "intro": make_pattern(
        kick  = "................",
        snare = "................",
        hihat = "x...x...x...x...",
    ),
    "outro": make_pattern(
        kick  = "x...............",
        snare = "................",
        hihat = "x...x...x...x...",
    ),
}

print("Drum patterns by section type:")
for kind in ["intro", "verse", "chorus", "bridge", "outro"]:
    pattern = drum_patterns[kind]
    print(f"\n  === {kind.upper()} ===")
    for inst, hits in pattern.items():
        row = " ".join("X" if h else "." for h in hits[:16])  # show first 16
        print(f"  {inst:<8}  {row}")

## Step 5: Lyric Framework

In [ ]:
# === STEP 5: Lyric framework (Chapter 5) ===
# We define the structure (rhyme scheme + syllable targets) for each section.
# Actual lyrics would be written by the songwriter; we provide the skeleton.

import re

def count_syllables(word: str) -> int:
    word = word.lower().strip(".,!?;:\"'()")
    if not word:
        return 0
    vowel_groups = re.findall(r'[aeiouy]+', word)
    count = len(vowel_groups)
    if word.endswith('e') and count > 1:
        count -= 1
    if word.endswith('le') and len(word) > 2 and word[-3] not in 'aeiou':
        count += 1
    return max(1, count)

def count_line_syllables(line: str) -> int:
    return sum(count_syllables(w) for w in line.strip().split())

@dataclass
class LyricSection:
    name: str
    rhyme_scheme: str            # e.g. "ABAB"
    syllable_targets: list[int]  # target syllable count per line
    sample_lines: list[str]      # placeholder lyrics

lyric_framework: list[LyricSection] = [
    LyricSection("Verse 1", "ABAB", [8, 7, 8, 7], [
        "Walking through the city rain",
        "Neon lights are all I see",
        "Looking for a way to change",
        "Something that could set me free",
    ]),
    LyricSection("Chorus", "AABB", [6, 6, 8, 8], [
        "Light the spark tonight",
        "Set the world alight",
        "We can find a way to start again",
        "Leave behind the hurt and pain",
    ]),
    LyricSection("Verse 2", "ABAB", [8, 7, 8, 7], [
        "Shadows fade beneath the dawn",
        "Silence breaks with morning sound",
        "Yesterday is finally gone",
        "Tomorrow waits on solid ground",
    ]),
    LyricSection("Bridge", "AABB", [7, 7, 7, 7], [
        "Maybe we were lost before",
        "Maybe now we found the door",
        "Standing at the edge of change",
        "Nothing here will stay the same",
    ]),
]

print("LYRIC FRAMEWORK")
print("═" * 60)
for ls in lyric_framework:
    print(f"\n{ls.name} (rhyme: {ls.rhyme_scheme}, target syllables: {ls.syllable_targets})")
    print("─" * 50)
    for i, line in enumerate(ls.sample_lines):
        actual_syl = count_line_syllables(line)
        target = ls.syllable_targets[i]
        match = "✓" if abs(actual_syl - target) <= 1 else "!"
        print(f"  {ls.rhyme_scheme[i]}: ({actual_syl}/{target} syl) {match}  {line}")

## Step 6: Arrangement

In [ ]:
# === STEP 6: Arrangement plan (Chapter 6) ===

instruments: list[str] = ["Vocals", "Acoustic Gtr", "Electric Gtr", "Bass", "Drums", "Keys"]

# Intensity per instrument per section (0 = silent, 1 = full)
song_arrangement: dict[str, dict[str, float]] = {
    "Intro":     {"Vocals": 0.0, "Acoustic Gtr": 0.5, "Electric Gtr": 0.0, "Bass": 0.0, "Drums": 0.0, "Keys": 0.4},
    "Verse 1":   {"Vocals": 0.6, "Acoustic Gtr": 0.6, "Electric Gtr": 0.0, "Bass": 0.4, "Drums": 0.3, "Keys": 0.3},
    "Chorus 1":  {"Vocals": 0.9, "Acoustic Gtr": 0.7, "Electric Gtr": 0.5, "Bass": 0.7, "Drums": 0.7, "Keys": 0.5},
    "Verse 2":   {"Vocals": 0.6, "Acoustic Gtr": 0.5, "Electric Gtr": 0.2, "Bass": 0.5, "Drums": 0.4, "Keys": 0.3},
    "Chorus 2":  {"Vocals": 1.0, "Acoustic Gtr": 0.7, "Electric Gtr": 0.7, "Bass": 0.8, "Drums": 0.8, "Keys": 0.6},
    "Bridge":    {"Vocals": 0.7, "Acoustic Gtr": 0.3, "Electric Gtr": 0.0, "Bass": 0.3, "Drums": 0.2, "Keys": 0.7},
    "Chorus 3":  {"Vocals": 1.0, "Acoustic Gtr": 0.8, "Electric Gtr": 1.0, "Bass": 1.0, "Drums": 1.0, "Keys": 0.7},
    "Outro":     {"Vocals": 0.3, "Acoustic Gtr": 0.4, "Electric Gtr": 0.0, "Bass": 0.2, "Drums": 0.1, "Keys": 0.3},
}

# Compute dynamics per section
sections_list = [s.name for s in song_structure]
dynamics = []
for section_name in sections_list:
    total = sum(song_arrangement[section_name].values())
    dynamics.append(total / len(instruments))

print("ARRANGEMENT OVERVIEW")
print("═" * 60)
for i, section in enumerate(song_structure):
    arr = song_arrangement[section.name]
    active = [inst for inst, val in arr.items() if val > 0]
    print(f"  {section.name:<12} Dynamic: {dynamics[i]:.2f}  Active: {', '.join(active)}")

## Step 7: The Complete Song Map

This is the culmination -- a single visualization that shows **every dimension** of our song.

In [ ]:
# === STEP 7: Complete song map visualization ===

fig = plt.figure(figsize=(16, 14))
fig.suptitle(f"Complete Song Map: Key of {KEY} Major, {TEMPO} BPM",
             fontsize=16, fontweight="bold", y=0.98)

# Create grid layout: 4 rows
gs = fig.add_gridspec(4, 1, height_ratios=[1, 2, 1.5, 1], hspace=0.35)

# --- Row 1: Song Structure Timeline ---
ax1 = fig.add_subplot(gs[0])
section_colors = {
    "intro": "#A8D8EA", "verse": "#AA96DA", "chorus": "#FCBAD3",
    "bridge": "#FFFFD2", "outro": "#B5EAD7",
}

x = 0
for section in song_structure:
    color = section_colors[section.kind]
    ax1.barh(0, section.bars, left=x, height=0.6, color=color,
             edgecolor="white", linewidth=2)
    # Chord names for this section
    chord_str = ", ".join([all_diatonic[d][1] for d in section.chord_degrees[:4]])
    ax1.text(x + section.bars / 2, 0.05, section.name,
            ha="center", va="center", fontsize=8, fontweight="bold")
    ax1.text(x + section.bars / 2, -0.15, chord_str,
            ha="center", va="center", fontsize=6, color="gray")
    x += section.bars

ax1.set_xlim(0, total_bars)
ax1.set_ylim(-0.5, 0.5)
ax1.set_yticks([])
ax1.set_title("Song Structure + Chord Progressions", fontsize=11, loc="left")
ax1.spines["top"].set_visible(False)
ax1.spines["right"].set_visible(False)
ax1.spines["left"].set_visible(False)

# --- Row 2: Melody Contour ---
ax2 = fig.add_subplot(gs[1])
bar_position = 0
for section in song_structure:
    melody = section_melodies[section.name]
    n_notes = len(melody)
    # Map notes to bar positions
    notes_per_bar = n_notes / section.bars
    x_positions = [bar_position + i / notes_per_bar for i in range(n_notes)]
    
    color = section_colors[section.kind]
    ax2.plot(x_positions, melody, "-", color=color, linewidth=1.5, alpha=0.9)
    ax2.scatter(x_positions, melody, color=color, s=8, zorder=3)
    bar_position += section.bars

ax2.set_xlim(0, total_bars)
ax2.set_ylabel("MIDI Pitch")
ax2.set_title("Melodic Contour", fontsize=11, loc="left")
ax2.grid(True, alpha=0.2)
ax2.spines["top"].set_visible(False)
ax2.spines["right"].set_visible(False)

# --- Row 3: Arrangement Heatmap ---
ax3 = fig.add_subplot(gs[2])
matrix = np.zeros((len(instruments), len(sections_list)))
for j, section_name in enumerate(sections_list):
    for i, inst in enumerate(instruments):
        matrix[i, j] = song_arrangement[section_name][inst]

cmap_arr = LinearSegmentedColormap.from_list("arr", ["#FFFFFF", "#E8E0F0", "#AA96DA", "#7B5EA7"])
im = ax3.imshow(matrix, aspect="auto", cmap=cmap_arr, vmin=0, vmax=1)
ax3.set_xticks(range(len(sections_list)))
ax3.set_xticklabels([s.name for s in song_structure], fontsize=8, rotation=20, ha="right")
ax3.set_yticks(range(len(instruments)))
ax3.set_yticklabels(instruments, fontsize=9)
ax3.set_title("Arrangement: Instrument Intensity", fontsize=11, loc="left")
for i in range(len(instruments)):
    for j in range(len(sections_list)):
        val = matrix[i, j]
        if val > 0:
            tc = "white" if val > 0.6 else "#333"
            ax3.text(j, i, f"{val:.1f}", ha="center", va="center", fontsize=7, color=tc)

# --- Row 4: Dynamics Curve ---
ax4 = fig.add_subplot(gs[3])
section_mids = []
bar_pos = 0
for section in song_structure:
    section_mids.append(bar_pos + section.bars / 2)
    bar_pos += section.bars

ax4.fill_between(section_mids, dynamics, alpha=0.3, color="#FCBAD3")
ax4.plot(section_mids, dynamics, "o-", color="#FCBAD3", linewidth=2.5, markersize=8)
for xp, dyn in zip(section_mids, dynamics):
    ax4.text(xp, dyn + 0.03, f"{dyn:.2f}", ha="center", fontsize=8)

ax4.set_xlim(0, total_bars)
ax4.set_ylim(0, 1.0)
ax4.set_xlabel("Bars")
ax4.set_ylabel("Energy")
ax4.set_title("Dynamics Curve", fontsize=11, loc="left")
ax4.spines["top"].set_visible(False)
ax4.spines["right"].set_visible(False)

# Legend for section colors
legend_patches = [mpatches.Patch(color=c, label=k.title()) for k, c in section_colors.items()]
fig.legend(handles=legend_patches, loc="lower center", ncol=5, fontsize=9)

plt.tight_layout(rect=[0, 0.04, 1, 0.96])
plt.show()

## Listening to the Song

Let's render a simplified version: melody over chords, with drums.

In [ ]:
# === Audio synthesis: combine melody + chords + drums ===

SAMPLE_RATE = 22050

def synth_note(freq: float, duration: float, sr: int = SAMPLE_RATE) -> np.ndarray:
    """Synthesize a single note with envelope."""
    t = np.linspace(0, duration, int(sr * duration), endpoint=False)
    wave = np.sin(2 * np.pi * freq * t)
    # Add a bit of the 2nd harmonic for warmth
    wave += 0.3 * np.sin(2 * np.pi * freq * 2 * t)
    # Envelope
    fade = int(len(t) * 0.08)
    env = np.ones(len(t))
    env[:fade] = np.linspace(0, 1, fade)
    env[-fade:] = np.linspace(1, 0, fade)
    return wave * env * 0.3

def synth_chord(midi_notes: list[int], duration: float, sr: int = SAMPLE_RATE) -> np.ndarray:
    """Synthesize a chord (multiple notes)."""
    wave = sum(synth_note(midi_to_freq(n), duration, sr) for n in midi_notes)
    return wave / len(midi_notes) * 0.5

def synth_kick(sr: int = SAMPLE_RATE) -> np.ndarray:
    t = np.linspace(0, 0.15, int(sr * 0.15), endpoint=False)
    freq = 150 * np.exp(-t * 20) + 50
    return np.sin(np.cumsum(2 * np.pi * freq / sr)) * np.exp(-t * 15) * 0.6

def synth_snare(sr: int = SAMPLE_RATE) -> np.ndarray:
    t = np.linspace(0, 0.1, int(sr * 0.1), endpoint=False)
    rng = np.random.default_rng(42)
    return (rng.normal(0, 1, len(t)) * np.exp(-t * 30) * 0.3 +
            np.sin(2 * np.pi * 200 * t) * np.exp(-t * 35) * 0.2)

def synth_hihat(sr: int = SAMPLE_RATE) -> np.ndarray:
    t = np.linspace(0, 0.04, int(sr * 0.04), endpoint=False)
    rng = np.random.default_rng(7)
    return rng.normal(0, 1, len(t)) * np.exp(-t * 80) * 0.15

# Render the full song
beat_duration = 60.0 / TEMPO  # seconds per beat
bar_duration = beat_duration * TIME_SIG[0]  # seconds per bar
sixteenth_duration = beat_duration / 4

total_samples = int(total_bars * bar_duration * SAMPLE_RATE)
song_audio = np.zeros(total_samples)

# Pre-compute drum sounds
kick_sound = synth_kick()
snare_sound = synth_snare()
hihat_sound = synth_hihat()

bar_offset = 0  # running bar counter

for section in song_structure:
    melody = section_melodies[section.name]
    notes_per_bar = len(melody) // section.bars
    note_dur = bar_duration / notes_per_bar
    
    arr_intensity = song_arrangement[section.name]
    
    for bar_i in range(section.bars):
        bar_start_sec = (bar_offset + bar_i) * bar_duration
        
        # --- Melody ---
        if arr_intensity["Vocals"] > 0:
            for note_i in range(notes_per_bar):
                idx = bar_i * notes_per_bar + note_i
                if idx < len(melody):
                    note_start = bar_start_sec + note_i * note_dur
                    sample_start = int(note_start * SAMPLE_RATE)
                    note_audio = synth_note(midi_to_freq(melody[idx]), note_dur * 0.9)
                    note_audio *= arr_intensity["Vocals"]
                    end = min(sample_start + len(note_audio), total_samples)
                    song_audio[sample_start:end] += note_audio[:end - sample_start]
        
        # --- Chords (one per bar) ---
        chord_deg = section.chord_degrees[bar_i % len(section.chord_degrees)]
        chord_notes = all_diatonic[chord_deg][2]
        chord_intensity = max(arr_intensity.get("Acoustic Gtr", 0), arr_intensity.get("Keys", 0))
        if chord_intensity > 0:
            sample_start = int(bar_start_sec * SAMPLE_RATE)
            chord_audio = synth_chord(chord_notes, bar_duration * 0.95)
            chord_audio *= chord_intensity * 0.6
            end = min(sample_start + len(chord_audio), total_samples)
            song_audio[sample_start:end] += chord_audio[:end - sample_start]
        
        # --- Drums ---
        drum_intensity = arr_intensity["Drums"]
        if drum_intensity > 0:
            drum_pat = drum_patterns[section.kind]
            for slot in range(16):
                slot_start = bar_start_sec + slot * sixteenth_duration
                sample_pos = int(slot_start * SAMPLE_RATE)
                
                if drum_pat["Kick"][slot % len(drum_pat["Kick"])]:
                    end = min(sample_pos + len(kick_sound), total_samples)
                    song_audio[sample_pos:end] += kick_sound[:end - sample_pos] * drum_intensity
                if drum_pat["Snare"][slot % len(drum_pat["Snare"])]:
                    end = min(sample_pos + len(snare_sound), total_samples)
                    song_audio[sample_pos:end] += snare_sound[:end - sample_pos] * drum_intensity
                if drum_pat["Hi-hat"][slot % len(drum_pat["Hi-hat"])]:
                    end = min(sample_pos + len(hihat_sound), total_samples)
                    song_audio[sample_pos:end] += hihat_sound[:end - sample_pos] * drum_intensity
    
    bar_offset += section.bars

# Normalize
peak = np.max(np.abs(song_audio))
if peak > 0:
    song_audio = song_audio / peak * 0.85

print(f"Song rendered: {len(song_audio) / SAMPLE_RATE:.1f} seconds")
print(f"Key: {KEY} major | Tempo: {TEMPO} BPM | Bars: {total_bars}")
Audio(song_audio, rate=SAMPLE_RATE)

## The Lyric Sheet

Let's print the complete lyric sheet with chords and structure annotations.

In [ ]:
# === Print the final song sheet ===

print("═" * 50)
print("          COMPLETE SONG SHEET")
print(f"     Key: {KEY} major | Tempo: {TEMPO} BPM")
print("═" * 50)

# Map lyric sections to structure
lyric_map = {ls.name: ls for ls in lyric_framework}

for section in song_structure:
    print(f"\n[{section.name}]")
    chords = [all_diatonic[d][1] for d in section.chord_degrees[:4]]
    print(f"  Chords: | {' | '.join(chords)} |")
    
    # Find matching lyric section
    # Verse 1/2 share the same verse framework, Chorus 1/2/3 share chorus
    lyric_key = None
    if "Verse 1" in section.name:
        lyric_key = "Verse 1"
    elif "Verse 2" in section.name:
        lyric_key = "Verse 2"
    elif "Chorus" in section.name:
        lyric_key = "Chorus"
    elif "Bridge" in section.name:
        lyric_key = "Bridge"
    
    if lyric_key and lyric_key in lyric_map:
        ls = lyric_map[lyric_key]
        for i, line in enumerate(ls.sample_lines):
            print(f"  {line}")
    elif section.kind == "intro":
        print("  (instrumental)")
    elif section.kind == "outro":
        print("  (fade out)")

print("\n" + "═" * 50)

## Summary: What We Built

In this chapter we combined every concept from the course:

| Step | Chapter | What We Did |
|------|---------|-------------|
| 1 | Ch 1 | Chose a verse-chorus form with 8 sections (56 bars) |
| 2 | Ch 3 | Assigned diatonic chord progressions to each section |
| 3 | Ch 2 | Generated melodies from the G major scale, shaped to the chords |
| 4 | Ch 4 | Created drum patterns with varying density per section |
| 5 | Ch 5 | Built a lyric framework with rhyme schemes and syllable targets |
| 6 | Ch 6 | Designed an arrangement with instruments entering gradually |
| 7 | All  | Rendered everything as audio and visualized the complete song map |

## Where to Go From Here

This course gave you a **computational foundation** for songwriting. The tools here are starting points -- real songwriting requires ear, taste, and practice that no algorithm can replace. But understanding these structures will help you:

1. **Analyze** songs you admire -- break them down into their components
2. **Generate ideas** -- use algorithmic tools to spark creativity, then refine by ear
3. **Diagnose problems** -- if a song does not work, check structure, dynamics, prosody
4. **Communicate** -- talk to other musicians using precise vocabulary

The best songs come from combining technical knowledge with genuine emotion. Now you have the technical side. Go write something that matters to you.